In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime, timedelta

# We will explore statistics looking at these packages
import pandas as pd
import seaborn as sn
import statsmodels.api as sm

### Datasets

We will look at two datasets. First the [Goddard Institute for Space Studies surface temperature analysis dataset from NASA](https://data.giss.nasa.gov/gistemp). There are many intersting products here; we will look at zonal mean temperature anomalies. The 'base' period that defines the mean is 1951 through 1980.

In [ ]:
url = 'https://data.giss.nasa.gov/gistemp/tabledata_v3/ZonAnn.Ts+dSST.csv'
df_zonal = pd.read_csv(url, index_col=0, parse_dates=True)

# The `describe` method is a good way to get an idea of the statistical
# properties of a variable or dataset.
df_zonal['Glob'].describe()

In [ ]:
df_zonal.describe()

The second dataset will be [monthly mean temperature data for states](https://www1.ncdc.noaa.gov/pub/data/cirs/climdiv/climdiv-tmpcst-v1.0.0-20170404) published by NOAA. The [README file for this dataset](https://www1.ncdc.noaa.gov/pub/data/cirs/climdiv/state-readme.txt) describes the mangled time, location, and category code in the first column. They also have a [wide range of data available](https://www1.ncdc.noaa.gov/pub/data/cirs/climdiv/).




In [ ]:
# Region IDs according to the README file. A new first value 'null' has been
# added so that the index of the list will corresond to the code (i.e., 'Alabama' is 001)
# We will only deal with the contiguous 48, so other regions are ignored.
Region_ID = ['null', 'Alabama', 'Arizona', 'Arkansas', 'California', 
             'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Idaho', 
             'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 
             'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 
             'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 
             'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 
             'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 
             'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 
             'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [ ]:
import urllib.request # use this package to read html files as text

url = 'https://www1.ncdc.noaa.gov/pub/data/cirs/climdiv/climdiv-tmpcst-v1.0.0-20170404'
with urllib.request.urlopen(url) as f:
   html = f.read()  # the entire file as a single string

lines = html.split(b'\n')  # split into lines. The 'b' is needed since the 'string' is actually
                           # defined as a 'bytes' object, that could contain special characters


In [ ]:
times = {state:[] for state in Region_ID}
monthly_temp = {state:[] for state in Region_ID}

for line in lines:
    data = line.split()
    if not data: 
        continue
    
    # First parse the first element into state, division, element, and year according 
    # to the README file
    state_code = int(data[0][:3])
    if state_code > 48:  
        continue # ignore regions outside the contiguous 48.
    state = Region_ID[state_code]
    division = int(data[0][3])  # Zero for area-averaged element. We won't use this
    element = int(data[0][4:6]) # Should all be 02 (average temperature) for this dataset
    year = int(data[0][6:])
    
    monthly_temp[state] += [float(temp) for temp in data[1:]]
    times[state] += [datetime(year, month, 15) for month in range(1, 13)]

dfs = [pd.DataFrame(monthly_temp[state], index=times[state], columns=[state])
       for state in Region_ID]

df_states = pd.concat(dfs[1:], axis=1) # concatinate and remove the 'null' state
df_states[df_states == -99.9] = np.nan

# Remove 1951 - 1980 mean to get anomalies similar to GISS data.
df_states -= df_states['1951':'1980'].mean()


In [ ]:
df_states

In [ ]:
df_states['Texas'].plot()

In [ ]:
df_states.resample('AS').mean()['Texas'].plot()

In [ ]:
df = pd.concat([df_zonal, df_states.resample('AS').mean()['Texas']], axis=1).dropna()

In [ ]:
df.describe()

In [ ]:
# We can quickly calculate a correlation matrix between all the columns
df.corr()

We are now ready to start doing some statistics. Fitting a model in statsmodels typically involves 3 easy steps:

1. Use the model class to describe the model
2. Fit the model using a class method
3. Inspect the results using a summary method


In [ ]:
# 1. Describe the model
mod = sm.OLS(df['Texas'], df.drop('Texas', axis=1))   # Ordinary Least Squares

# 2. Fit the model
res = mod.fit()

# 3. Summarize the model fit
res.summary()

In [ ]:
res.pvalues  # lower is better. Significant usually for p < 0.05

### Visualizing datasets

We can use seaborn to visulize joint dataset distributions

In [ ]:
sn.jointplot(df['Texas'], df['24N-44N'])

In [ ]:
# Seaborn also plays nice with pandas -- you can provide the dataframe as 'data'
# and then just reference the columns.
# Excercise -- Try kind as 'reg', 'kde', and 'hex'
sn.jointplot('Texas', '24N-44N', data=df, kind='reg')

In [ ]:
# We can quicly visualize a 'heatmap' of the correlation coefficients.
sn.heatmap(df.corr())

In [ ]:
fig = plt.figure(figsize=(12, 4))
sn.boxplot(df)
# sn.violinplot(df.drop('Texas', axis=1))

In [ ]:
df[['Glob', 'NHem', 'SHem', 'Texas']].plot(figsize=(15, 5))

In [ ]:
import statsmodels.formula.api as smf

model = smf.ols(formula="Texas ~ NHem + SHem", data=df).fit()
model.summary()

In [ ]:
df['year'] = df.index.year
model = smf.ols(formula="Texas ~ year", data=df).fit()
model.summary()

In [ ]:
sm.stats.anova_lm(model)